In [5]:
# # ──────────────────────────────────────────────────────────────────────────────
# # CELDA 0 · INSTALACIÓN DE DEPENDENCIAS
# # '!' ejecuta en la shell desde el notebook (instalación en el kernel actual).
# # - python-telegram-bot==21.10 → librería oficial (API asíncrona moderna).
# # - python-dotenv → opcional si luego quieres cargar variables desde un archivo .env.
# # El flag --upgrade asegura versión solicitada; -q baja el ruido de salida.
# # ──────────────────────────────────────────────────────────────────────────────
# !pip install -q --upgrade python-telegram-bot==21.10 python-dotenv==1.*


In [6]:
# ──────────────────────────────────────────────────────────────────────────────
# CELDA 1 · VERIFICACIÓN DEL ENTORNO
# Importamos módulos útiles para diagnosticar:
# - sys: info del intérprete (ruta, versión).
# - sysconfig / site: rutas donde se instalan paquetes (site-packages).
# - telegram: probamos que la librería está instalada e importable.
# - subprocess: para invocar 'pip --version' con el MISMO intérprete del kernel.
# - pkgutil: utilidades sobre paquetes (no estrictamente necesario aquí).
# ──────────────────────────────────────────────────────────────────────────────
import sys, sysconfig, site, telegram, subprocess, pkgutil

# sys.executable → ruta del binario de Python que usa este kernel.
print("Executable:", sys.executable)

# site.getsitepackages() → carpetas donde pip coloca paquetes para este entorno.
print("Site-packages:", site.getsitepackages())

# telegram.__version__ → versión de python-telegram-bot cargada.
print("python-telegram-bot:", telegram.__version__)

# sys.version → versión exacta de Python (útil para compatibilidad).
print("Python:", sys.version)



# Llamamos a 'pip --version' usando el mismo Python del kernel (evita confusiones).
print("pip:", subprocess.check_output([sys.executable, "-m", "pip", "--version"]).decode())

# Esperado:
# - python-telegram-bot: 21.10
# - Python: 3.9+ (en tu caso 3.13.x)


Executable: c:\Users\34680\AppData\Local\Programs\Python\Python313\python.exe
Site-packages: ['c:\\Users\\34680\\AppData\\Local\\Programs\\Python\\Python313', 'c:\\Users\\34680\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages']
python-telegram-bot: 21.10
Python: 3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]
pip: pip 25.0.1 from c:\Users\34680\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip (python 3.13)



In [ ]:

# ──────────────────────────────────────────────────────────────────────────────
# CELDA 2 · TOKEN DEL BOT
# Guardamos el token de @BotFather en una variable de entorno.
# Ventajas:
#   - No "quemamos" secretos en código (mejor práctica).
#   - Resto del código lee TOKEN desde os.environ["TELEGRAM_TOKEN"].
# Seguridad:
#   - ⚠️ Regenera tu token si alguna vez se expuso.
#   - No subas tokens a Git.
# ──────────────────────────────────────────────────────────────────────────────
import os

TOKEN = ""  # ← Sustituye por el token NUEVO (regenerado en BotFather)
os.environ["TELEGRAM_TOKEN"] = TOKEN  # Guardamos el token en el entorno del kernel

# bool(...) para comprobar que existe y no está vacío.
print("TOKEN cargado:", os.getenv("TELEGRAM_TOKEN"))


TOKEN cargado: 7864048279:AAG3icMDnrG_xPU4D6M5Z77CxtXNs27z4CY


In [ ]:
# app.py
from dotenv import load_dotenv
import os
from telegram.ext import ApplicationBuilder

load_dotenv()
TOKEN = os.getenv("TELEGRAM_TOKEN")
if not TOKEN:
    raise RuntimeError("Falta TELEGRAM_TOKEN en .env")
app = ApplicationBuilder().token(TOKEN).build()


In [8]:
# ──────────────────────────────────────────────────────────────────────────────
# CELDA 3 · PRUEBA DEL TOKEN (getMe)
# Creamos una Application mínima SOLO para verificar el token mediante bot.get_me().
# Como la librería usa asyncio, necesitamos:
#   - await app.initialize() para preparar la app.
#   - await app.bot.get_me() para consultar datos del bot a Telegram.
#   - await app.shutdown() para liberar recursos.
# En notebooks ya existe un event loop → usamos create_task si es posible.
# ──────────────────────────────────────────────────────────────────────────────
import asyncio
from telegram.ext import Application
import os

async def check_token():
    # Builder pattern: inyectamos el token desde el entorno.
    app = Application.builder().token(os.environ["TELEGRAM_TOKEN"]).build()
    await app.initialize()         # Inicializa recursos internos de la app/bot.
    me = await app.bot.get_me()    # Llamada a la API de Telegram para obtener info del bot.
    print("Bot OK ✅")
    print("id:", me.id, "| username:", f"@{me.username}", "| name:", me.first_name)
    await app.shutdown()           # Cerramos para no dejar recursos abiertos.

try:
    # Si hay event loop activo (notebook), lanzamos la coroutine como tarea.
    asyncio.get_running_loop()
    _ = asyncio.create_task(check_token())
except RuntimeError:
    # Si no hay loop (script .py), ejecutamos la coroutine directamente.
    asyncio.run(check_token())


INFO:httpx:HTTP Request: POST https://api.telegram.org/bot7864048279:AAG3icMDnrG_xPU4D6M5Z77CxtXNs27z4CY/getMe "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.telegram.org/bot7864048279:AAG3icMDnrG_xPU4D6M5Z77CxtXNs27z4CY/getMe "HTTP/1.1 200 OK"


Bot OK ✅
id: 7864048279 | username: @Revoquexbot | name: Xulzaz9bot


In [ ]:
import logging, os
from telegram.ext import Application
from telegram.ext import CommandHandler

# Celda 4 · APP Y HANDLERS BÁSICOS — Explicación línea por línea

# 1) Configuración básica
# - se configura el logging para ver info útil durante la ejecución.
# - TOKEN se lee desde las variables de entorno (fue puesto en la Celda 2).
# - assert asegura que haya token, si falta se detiene aquí.
logging.basicConfig(level=logging.INFO)
TOKEN = os.getenv("TELEGRAM_TOKEN")
assert TOKEN, "Falta TELEGRAM_TOKEN en el entorno (vuelve a la Celda 2)"

# 2) Definición de handlers (funciones que responden a comandos)
# - Son funciones async porque python-telegram-bot v20+ usa asyncio.
# - start: responde al comando /start con un saludo personalizado.
# - help_cmd: responde al comando /help con un texto corto.
# 'update' contiene el mensaje y el usuario; 'context' contiene utilidades del bot.
async def start(update, context):
    await update.message.reply_text(f"Hola {update.effective_user.first_name} 👋 ¡Bot en marcha!")

async def help_cmd(update, context):
    await update.message.reply_text("/start • /help")

# 3) Creación de la Application (núcleo del bot)
# - Application.builder().token(TOKEN).build() crea el objeto 'app' que gestiona
#   dispatcher, handlers, job_queue, sesiones HTTP, etc.
app = Application.builder().token(TOKEN).build()

# 4) Registro de handlers
# - CommandHandler("start", start) → cuando llegue /start se ejecuta start()
# - CommandHandler("help", help_cmd) → cuando llegue /help se ejecuta help_cmd()
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("help", help_cmd))

# 5) Nota final
# - La Celda 4 solo define y registra handlers; no arranca el bot.
#   Para ponerlo en marcha hay que iniciar polling o configurar webhook
#   (en este notebook se arranca polling en la Celda 5).
print("✅ App creada y handlers registrados.")

In [10]:
# ──────────────────────────────────────────────────────────────────────────────
# CELDA 5 · ARRANCAR POLLING
# Polling = el bot consulta periódicamente a Telegram por nuevos updates.
# En un notebook:
#   - Ya hay un event loop (asyncio) corriendo.
#   - Lanzamos el bot como "tarea de fondo" (create_task) y lo dejamos escuchando.
# Importante:
#   - Webhook y polling SON EXCLUYENTES. Si tenías webhook activo, bórralo.
#   - Mantenemos un 'polling_task' global para poder detenerlo después.
# ──────────────────────────────────────────────────────────────────────────────
import asyncio

polling_task = None  # Referencia global a la tarea en ejecución (si la hay)

async def _run_polling():
    # 1) Inicializa componentes internos (dispatcher, bot, etc.)
    await app.initialize()

    # 2) Elimina webhook si existiera (evita '409 Conflict' con getUpdates).
    try:
        # drop_pending_updates=True → descarta updates encolados en el servidor.
        await app.bot.delete_webhook(drop_pending_updates=True)
    except Exception:
        # Si no había webhook, ignoramos cualquier fallo aquí.
        pass

    # 3) Arranca la Application (abre conexiones, empieza job_queue si existiera, etc.)
    await app.start()

    # 4) Empieza el polling (la librería abrirá un loop interno para getUpdates).
    await app.updater.start_polling()

    print("🤖 Bot en polling… (usa la Celda 6 para detener)")

    # 5) Mantenemos viva la coroutine con un "latido" sencillo.
    #    En un script, no haría falta; en notebooks, previene que la rutina finalice.
    while True:
        await asyncio.sleep(5)

# Ejecutamos la coroutine según el contexto:
try:
    # Si el kernel ya tiene loop, creamos una tarea en ese loop (no bloquea el notebook).
    loop = asyncio.get_running_loop()
    polling_task = loop.create_task(_run_polling())
except RuntimeError:
    # Si no hay loop (script .py), ejecutamos de forma síncrona (bloqueante).
    asyncio.run(_run_polling())

# Esperado:
# - Ver el mensaje "🤖 Bot en polling…"
# - En Telegram, /start y /help deben responder.


In [11]:
# ──────────────────────────────────────────────────────────────────────────────
# CELDA 6 · DETENER POLLING
# Buenas prácticas antes de re-ejecutar la Celda 5:
#   - Cancelar la tarea asíncrona si sigue viva (polling_task.cancel()).
#   - Llamar a stop/shutdown para cerrar conexiones de forma ordenada.
# Notas:
#   - En Jupyter, si re-ejecutas Celda 5 sin parar antes, puedes crear tareas
#     duplicadas y/o ver errores de conflicto con getUpdates.
# ──────────────────────────────────────────────────────────────────────────────
import contextlib
import asyncio

async def _stop_polling():
    print("Deteniendo polling…")
    # app.updater.stop() → detiene las peticiones de getUpdates (polling).
    # app.stop() → detiene la aplicación (cierra bot internamente).
    # app.shutdown() → libera recursos (dispatcher, session, etc.).
    with contextlib.suppress(Exception):  # si ya estaba parado, no queremos stacktrace
        await app.updater.stop()
        await app.stop()
        await app.shutdown()
    print("Bot detenido ✅")

try:
    # Si existe una tarea de polling en ejecución y no ha terminado → cancélala.
    if 'polling_task' in globals() and polling_task and not polling_task.done():
        polling_task.cancel()

    # Ejecuta la parada según haya event loop (notebook) o no (script).
    try:
        asyncio.get_running_loop()
        asyncio.create_task(_stop_polling())   # en notebook, lanzar como tarea
    except RuntimeError:
        asyncio.run(_stop_polling())           # en script, ejecutar directamente
except NameError:
    # Si 'polling_task' no existe (no arrancaste Celda 5), informamos.
    print("No hay polling activo.")


Deteniendo polling…
Bot detenido ✅
